In [1]:
# import statements
import import_ipynb
import pandas as pd
from format_data import LoadAndFormatData
from bin_portfolio import Bin_Portfolio

importing Jupyter notebook from format_data.ipynb
importing Jupyter notebook from bin_portfolio.ipynb


In [2]:
# Load and modify the data
df = LoadAndFormatData()
df.head()

,A2ME,BEME,BEME_adj,BEME_ind,CEI,DATE,DEC_ME,DEC_SHROUT,Div,Div_ann,...,sale,sale_ind,sales_g,sga2s,spread_mean,std_turn,std_vol,suv,RET_t+1,mkt
0,NaN,NaN,NaN,0.824752,NaN,1986-01-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.257143,0.009116
1,NaN,NaN,NaN,0.825905,NaN,1986-02-28,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.076998,0.212080,7804.560824,NaN,0.365385,0.009116
2,NaN,NaN,NaN,0.825905,NaN,1986-03-31,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.055511,0.107977,3973.567637,-0.363314,-0.098592,0.009116
3,NaN,NaN,NaN,0.825015,NaN,1986-04-30,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.037231,0.174533,6422.825870,-0.571056,-0.222656,0.009116
4,NaN,NaN,NaN,0.824672,NaN,1986-05-31,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.048336,0.150228,5527.502665,-0.708679,-0.005025,0.009116


In [3]:
# Run OLS and determine the coefficients
subset = df[df['DATE'] == '2023-01-31']
subset.head()

,A2ME,BEME,BEME_adj,BEME_ind,CEI,DATE,DEC_ME,DEC_SHROUT,Div,Div_ann,...,sale,sale_ind,sales_g,sga2s,spread_mean,std_turn,std_vol,suv,RET_t+1,mkt
4033,0.372175,0.300876,-0.205358,0.506234,0.037643,2023-01-31,3.015299e+06,19089.0,0.0,2.665999,...,1144.579,365862.147,0.119899,0.197834,0.001713,0.384852,74003.690396,0.209738,-0.014585,0.034799
4536,0.540833,0.252106,-0.156684,0.408790,-4.884325,2023-01-31,1.095848e+05,26925.0,0.0,0.000000,...,140.966,2656305.584,0.237390,0.147539,0.011440,0.135287,36425.942427,-0.589648,0.092847,0.012912
5283,0.917032,0.385121,-0.067338,0.452459,-0.604233,2023-01-31,2.684632e+06,27997.0,0.0,0.000000,...,3368.865,835692.186,-0.006350,0.042673,0.001185,0.848834,234706.222689,-0.297358,-0.001042,0.071634
7037,0.514566,0.391178,-0.115056,0.506234,0.289530,2023-01-31,4.848945e+04,6177.0,0.0,0.000000,...,23.481,365862.147,-0.262763,0.223074,0.012456,0.061457,3834.303907,0.422182,-0.068479,0.025283
9953,1.140592,0.881976,0.429517,0.452459,NaN,2023-01-31,5.066578e+04,11594.0,0.0,0.000000,...,184.115,835692.186,1.452512,0.051902,0.016740,0.117628,13742.587884,0.960779,-0.015717,-0.023426


In [8]:
# bin the data and examine the returns
#vars_list = ["mkt", "prof", "roic", "BEME", "ME", "p1_mktrf_beta"]
vars_list = ["mkt", "roic", "BEME", 'ME']
go_long = [True]*len(vars_list)
portfolio = Bin_Portfolio(dataframe=subset, vars_to_bin=vars_list, go_long=go_long)
portfolio

,PERMNO,DATE,mkt,roic,BEME,ME
0,16021.0,2023-01-31,0.185413,0.116128,1.617227,3.136006e+06
1,62341.0,2023-01-31,0.054161,0.113950,0.624413,6.035082e+06
2,63765.0,2023-01-31,0.064555,0.425396,0.538500,6.091756e+06
3,70092.0,2023-01-31,0.062813,0.086021,0.771674,3.229416e+06
4,75241.0,2023-01-31,0.083886,0.238608,0.560209,5.473093e+07
5,91531.0,2023-01-31,0.080689,0.119082,0.513741,9.032523e+06


In [9]:
# Evaluate the success of this portfolio based on next month's returns
portfolio = portfolio.merge(df[['PERMNO', 'DATE', 'RET_t+1']], on=["PERMNO", "DATE"], how="left")
portfolio

,PERMNO,DATE,mkt,roic,BEME,ME,RET_t+1
0,16021.0,2023-01-31,0.185413,0.116128,1.617227,3.136006e+06,-0.005520
1,62341.0,2023-01-31,0.054161,0.113950,0.624413,6.035082e+06,-0.009154
2,63765.0,2023-01-31,0.064555,0.425396,0.538500,6.091756e+06,-0.039855
3,70092.0,2023-01-31,0.062813,0.086021,0.771674,3.229416e+06,-0.078804
4,75241.0,2023-01-31,0.083886,0.238608,0.560209,5.473093e+07,-0.129976
5,91531.0,2023-01-31,0.080689,0.119082,0.513741,9.032523e+06,0.011795


In [5]:
# Sharpe ratio and PNL curve